### Add all the packages required 

In [1]:
import nipype.pipeline.engine as pe
import nipype.interfaces.fsl as fsl
import nipype.interfaces.io as nio

In [2]:
#!/usr/bin/env python
import argparse
import os
import sys
import pandas as pd
import patsy
import json
import numpy as np
from create_flame_model_files import create_flame_model_files
__version__ = 0.1

In [ ]:
###

In [3]:
working_dir = '/home/jovyan/work/test_data/'
file_list = '/home/jovyan/work/test_data/IBA_TRT/'
in_model_file = '/home/jovyan/work/test_data/IBA_TRT/model.json'
in_bids_dir =  '/home/jovyan/work/test_data/IBA_TRT/'

In [16]:
# load in the model
with open(in_model_file) as model_fd:
     model_dict = json.load(model_fd)

In [17]:
# parse the model string to determine which columns of the pheno
    # file we are interested in
in_columns = model_dict["model"].replace("-1", "").replace("-", "+").split("+")
t_columns = []
for column in in_columns:
    if '*' in column:
            t_columns += column.split("*")
    else:
            t_columns.append(column)
    in_columns = list(set(t_columns))

In [18]:
# read in the pheno file
pheno_df = pd.read_csv(os.path.join(in_bids_dir,'participants.tsv'), sep='\t')

# reduce the file to just the columns that we are interested in
#heno_df = pheno_df[['participant_id'] + in_columns]
pheno_df = pheno_df[['sub'] + in_columns]

# remove rows that have empty elements
pheno_df = pheno_df.dropna()

In [ ]:
file_list = 

In [10]:
import nipype.pipeline.engine as pe
import nipype.interfaces.fsl as fsl
import nipype.interfaces.io as nio

wf = pe.Workflow(name='wf_randomize')
wf.base_dir = working_dir

# First merge input files into single 4D file
merge = pe.Node(interface=fsl.Merge(), name='fsl_merge')
merge.inputs.in_files = file_list
merge.inputs.dimension = 't'
merge_output = "randomise_pipe_merge.nii.gz"
merge.inputs.merged_file = merge_output

# Create a mask from the merged file
mask = pe.Node(interface=fsl.maths.MathsCommand(), name='fsl_maths')
mask.inputs.args = '-abs -Tmin -bin'
merge_mask_output = "randomise_pipe_mask.nii.gz"
mask.inputs.out_file = merge_mask_output
wf.connect(merge, 'merged_file', mask, 'in_file')

TraitError: The 'in_files' trait of a MergeInputSpec instance must be a list of items which are an existing file name, but a value of '/home/jovyan/work/test_data/IBA_TRT/' <class 'str'> was specified.

In [5]:
randomise = pe.Node(interface=fsl.Randomise(), name='fsl_randomise')
wf.connect(mask, 'out_file', randomise, 'mask')
randomise.inputs.base_name = "randomise_pipe_contrast_{0}".format(current_contrast)
randomise.inputs.design_mat = mat_file
randomise.inputs.tcon = con_file
randomise.inputs.args = ' --skipTo={0}'.format(current_contrast)
randomise.inputs.num_perm = num_iterations
randomise.inputs.demean = True
randomise.inputs.tfce = True
wf.connect(merge, 'merged_file', randomise, 'in_file')

NameError: name 'wf' is not defined

In [ ]:
# We want to parallelize so that each contrast is processed
        # separately
        for current_contrast in range(1, num_contrasts + 1):
            # use randomize to use perform permutation test for contrast
            randomise = pe.Node(interface=fsl.Randomise(), name='fsl_randomise_{0}'.format(current_contrast))
            wf.connect(mask, 'out_file', randomise, 'mask')
            randomise.inputs.base_name = "randomise_pipe_contrast_{0}".format(current_contrast)
            randomise.inputs.design_mat = mat_file
            randomise.inputs.tcon = con_file
            randomise.inputs.args = ' --skipTo={0}'.format(current_contrast)
            randomise.inputs.num_perm = num_iterations
            randomise.inputs.demean = True
            randomise.inputs.tfce = True
            wf.connect(merge, 'merged_file', randomise, 'in_file')

            # threshold the resulting t corrected p file
            thresh = pe.Node(interface=fsl.Threshold(),
                             name='fsl_threshold_contrast_{0}'.format(current_contrast))
            thresh.inputs.thresh = 0.95
            wf.connect(randomise, "t_corrected_p_files", thresh, "in_file")

            # binarize the result of applying the threshold to get a mask
            thresh_bin = pe.Node(interface=fsl.maths.MathsCommand(),
                                 name='fsl_threshold_bin_contrast_{0}'.format(current_contrast))
            thresh_bin.inputs.args = '-bin'
            wf.connect(thresh, "out_file", thresh_bin, "in_file")

            # apply calculated mask to the statistic image
            apply_mask = pe.Node(interface=fsl.ApplyMask(),
                                 name='fsl_applymask_contrast_{0}'.format(current_contrast))
            wf.connect(randomise, 'tstat_files', apply_mask, 'in_file')
            wf.connect(thresh_bin, 'out_file', apply_mask, 'mask_file')

            # cluster the results to get a report of the findings
            cluster = pe.Node(interface=fsl.Cluster(),
                              name='cluster_contrast_{0}'.format(current_contrast))
            cluster.inputs.threshold = 0.0001
            cluster.inputs.out_index_file = "cluster_index_contrast_{0}".format(current_contrast)
            cluster.inputs.out_localmax_txt_file = "lmax_contrast_{0}.txt".format(current_contrast)
            cluster.inputs.out_size_file = "cluster_size_contrast_{0}".format(current_contrast)
            cluster.inputs.out_threshold_file = "randomise_out_contrast_{0}".format(current_contrast)
            cluster.inputs.terminal_output = 'file'
            wf.connect(apply_mask, 'out_file', cluster, 'in_file')

            # attach a datasink to save the output
            datasink = pe.Node(nio.DataSink(), name='sinker_contrast_{0}'.format(current_contrast))
            datasink.inputs.base_directory = output_dir

            wf.connect(apply_mask, 'out_file', datasink, 'output.@thresh_stat_file')
            wf.connect(cluster, 'index_file', datasink, 'output.@index_file')
            wf.connect(cluster, 'threshold_file', datasink, 'output.@threshold_file')
            wf.connect(cluster, 'localmax_txt_file', datasink, 'output.@localmax_txt_file')
            wf.connect(cluster, 'localmax_vol_file', datasink, 'output.@localmax_vol_file')
            wf.connect(cluster, 'max_file', datasink, 'output.@max_file')
            wf.connect(cluster, 'mean_file', datasink, 'output.@mean_file')
            wf.connect(cluster, 'pval_file', datasink, 'output.@pval_file')
            wf.connect(cluster, 'size_file', datasink, 'output.@size_file')

          